In [16]:
from typing import Dict, Optional 
from urllib.parse import urlparse
import re
import os
from googleapiclient.discovery import build
from datetime import datetime
import isodate
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

In [113]:
def message(text):
    try:
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {os.getenv('OPENROUTER_API_KEY')}",
            },
            data=json.dumps({
                # "model": "google/gemini-2.0-flash-exp:free",
                "model": "minimax/minimax-01",
                "messages": [
                    {
                        "role": "user",
                        "content": (text)
                    }
                ]
                
            })
        )
        if response.status_code == 200:
            data = response.json()
            return data['choices'][0]['message']['content']
        else:
            print(f"Error: {response.status_code}")
    except Exception as e:
        print(f"Error: {e}")

In [111]:
def llm_parser(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
    }
    response = requests.get(url, headers=headers)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    prompt = f"""Analyze this web content snippet and return JSON with:
    - title (most prominent heading)
    - author (personal or organizational)
    - date (prioritize publication dates in YYYY-MM-DD format)
    - source_organization (publisher/site owner)
    
    RESPOND WITH ONLY THE JSON AND NOTHING ELSE
    
    Content: {soup.prettify()}"""
    answer = message(prompt)
    print(answer)
    json_answer = parse_reference(answer)
    return json_answer

In [117]:
# link = "https://pubmed.ncbi.nlm.nih.gov/2315679/"
# DOESN'T WORKlink = "https://academic.oup.com/afraf/article/107/427/157/30448"
link = "https://permabond.com/products/cyanoacrylate-adhesives/"
llm_parser(link)

Response: <Response [200]>
```json
{
  "title": "About Cyanoacrylate Adhesives",
  "author": "Permabond",
  "date": "2025-01-04T07:00:30+00:00",
  "source_organization": "Permabond"
}
```


{'title': 'About Cyanoacrylate Adhesives',
 'author': 'Permabond',
 'date': '2025-01-04T07:00:30+00:00',
 'source_organization': 'Permabond'}

In [47]:
import json

def parse_reference(text):
    # Find the JSON content between the triple backticks
    json_start = text.find('```json\n') + 8  # Skip past ```json\n
    json_end = text.find('```', json_start)
    json_str = text[json_start:json_end].strip()
    
    # Parse the JSON string into a Python dictionary
    return json.loads(json_str)

# Example usage:
with open('response.txt', 'r') as f:
    content = f.read()
    reference_data = parse_reference(content)

# Now reference_data is a Python dictionary
print(reference_data['title'])  # Prints: Cosmological Significance of Time Reversal

Cosmological Significance of Time Reversal


In [118]:
from transformers import AutoTokenizer

# Initialize a tokenizer (GPT-2's tokenizer is commonly used for estimation)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Get the prettified HTML
html_text = soup.prettify()

# Count tokens
tokens = tokenizer.encode(html_text)
token_count = len(tokens)

print(f"Number of tokens: {token_count}")

Token indices sequence length is longer than the specified maximum sequence length for this model (56765 > 1024). Running this sequence through the model will result in indexing errors


Number of tokens: 56765
